In [ ]:
def timing(step):

    import pandas as pd
    import time
    
    start = time.time()

    ## Construct Dataframe

    import pandas as pd

    class DataController():
        dataMatrix = pd.DataFrame(columns=["title","desc","tag"])

        ## init will create dataMatrix
        def __init__(self, pathToFile, step):
            import os
            import json
            count = 0

            print('Begin loading')
            with open(pathToFile, 'r', encoding='utf-8') as fin:
                for line in fin:
                    ## for each line, add into dataMatrix, using ["title", "desc", "tag"] structure
                    line_dict = json.loads(line, encoding='utf-8')
                    self.dataMatrix = self.dataMatrix.append(line_dict, ignore_index=True)
                    count+=1
                    if count > step: break

        def getTrainingSet(self, label_class):
            ## classSet is set of data that has tag = label_class
            targetSet = self.dataMatrix[self.dataMatrix['tag']==label_class]
            restSet = self.dataMatrix[self.dataMatrix['tag']!=label_class]

            if(targetSet.shape[0] < restSet.shape[0]):
                # target has less population than the rest
                trainingSet = pd.concat([targetSet, restSet.sample(n=targetSet.shape[0])])
            else:
                # target has more population than the rest
                trainingSet = pd.concat([targetSet.sample(n=restSet.shape[0]), restSet])
            # shuffle data using sample fraction = 1
            trainingSet = trainingSet.sample(frac=1)
            return trainingSet

        def getData(self):
            return self.dataMatrix

    import os

    file_name = "masterDB_JPA Data - 20180406_flatten.json"
    #file_name = 'data20180620.json'
    file_path = os.getcwd()+"/../data/"+file_name

    data = DataController(file_path, step)
    ## Create vectorized data
    data = data.getData()
    vec_Desc = data['desc'] 
    vec_Title = data['title']

    ## create tokenizer
    import sys
    if not '..' in sys.path:
        sys.path.append('..')
    from tokenizer import Tokenizer
    import deepcut as dp
    def tokenizer(text):
        return dp.tokenize(text)
    #tkn1 = Tokenizer(1, dp.deepcut)
    tkn2 = Tokenizer(2, tokenizer)
    #tkn3 = Tokenizer(3, dp.deepcut)
    tkn4 = Tokenizer(4, tokenizer)

    ## open vocab file
    #import os
    #with open(os.path.abspath(os.path.join(os.getcwd(), '..', 'dict', 'desc_newdict_90p.txt'))  , 'rt', encoding='utf-8') as f_tv:
    #    desc_vocab = f_tv.read().split('\n')
    #with open(os.path.abspath(os.path.join(os.getcwd(), '..', 'dict', 'title_newdict_90p.txt'))  , 'rt', encoding='utf-8') as f_tv:
    #    title_vocab = f_tv.read().split('\n')

    ## create tfidf term-doc matrix
    from sklearn.feature_extraction.text import TfidfVectorizer

    desc_vectorizer = TfidfVectorizer(tokenizer=tkn2.tokenizer, min_df=0.85)
    desc_vectorizer.fit(vec_Desc)
    #desc_vec = desc_vectorizer.fit_transform(training_Title)

    title_vectorizer = TfidfVectorizer(tokenizer=tkn4.tokenizer, min_df=0.85)
    title_vectorizer.fit(vec_Title)
    #title_vec = title_vectorizer.fit_transform(training_Desc)
    
    return time.time() - start

In [ ]:
[print(timing(item)) for item in range(100, 1700, 100)]